In [3]:
import os
import pandas as pd

# ✅ Update this path to match your folder
IMAGE_FOLDER = r"C:\Users\mcsmu\Desktop\Autonomous A LAB\Dataset\teamD_dataset\Processed Dataset"
LABELS_FILE = r"C:\Users\mcsmu\Desktop\Autonomous A LAB\Dataset\teamD_dataset\labels.csv"

# Load the labels
labels_df = pd.read_csv(LABELS_FILE)
labels_df.head()


,00815481.png,0
0,00815521.png,1
1,00815561.png,0
2,00815602.png,0
3,00815643.png,0
4,00815684.png,0
